- #### Build a Question Answering application over a Graph Database

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

C:\Users\adity\AppData\Local\Temp\ipykernel_15912\3748794941.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)


##### Inserting the data

In [4]:
### Data set Movies

moview_query = """ 

LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE(m:Movie{id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, "|") | 
    MERGE(p: Person {name: trim(director)})
    MERGE(p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, "|") |
    MERGE(p: Person {name: trim(actor)})
    MERGE(p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, "|") |
    MERGE(g: Genre {name: trim(genre)})
    MERGE(m)-[:IN_GENRE]->(g))

"""

In [5]:
moview_query

' \n\nLOAD CSV WITH HEADERS FROM\n\'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv\' AS row\n\nMERGE(m:Movie{id: row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, "|") | \n    MERGE(p: Person {name: trim(director)})\n    MERGE(p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, "|") |\n    MERGE(p: Person {name: trim(actor)})\n    MERGE(p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, "|") |\n    MERGE(g: Genre {name: trim(genre)})\n    MERGE(m)-[:IN_GENRE]->(g))\n\n'

In [6]:
graph.query(moview_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {name: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


##### Quering with graph db

In [8]:
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027F0DA3A050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027F0DA3B4C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000027F69529ED0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027F0DA3A050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027F0DA3B4C0>, model_name=

In [10]:
response = chain.invoke({"query": "Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})-[r:DIRECTED]->(p:Person)RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': "I don't know the answer. \n"}

In [11]:
response = chain.invoke({"query": "Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})-[:ACTED_IN]->(p:Person) RETURN p.name

Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer. \n"}

In [12]:
response = chain.invoke({"query": "How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(p)  

Full Context:
[{'count(p)': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}